In [1]:
import requests
from bs4 import BeautifulSoup
from typing import List, Dict, Optional
import pandas as pd
from pydantic import BaseModel
from selenium import webdriver


In [3]:
## Ingredientai
resp = requests.get("https://www.15min.lt/maistas/receptas/varskes-ir-fermentinio-surio-pyragas-8796")
soup = BeautifulSoup(resp.content, "html.parser")
all_ingredients: List[Dict] = []

recipe_title = soup.find("div", class_="recipe-head").find("h1").text.strip()
recipe_image = soup.find("div", class_="image").find("img").get("src")
recipe_ingredients = soup.find("ul", class_="ingredients").find_all("li")
for ingredient in recipe_ingredients:
    the_ingredient = ingredient.find_all("span")
    all_ingredients.append({
        "ingredient_name": the_ingredient[0].text.strip(),
        "ingredient_amount": the_ingredient[2].text.strip(),
        "ingredient_note": the_ingredient[3].text.strip()
    })

#all_ingredients

## Paruosimo budas
recipe_manual: List[str] = []
recipe_making_steps = soup.find("div", class_="description text").find_all("p")

recipe_making_steps



for step in recipe_making_steps:
    stepsas = step.text.strip()
    recipe_manual.append(stepsas)

KeyboardInterrupt: 

Exception ignored in: '_pydevd_frame_eval.pydevd_frame_evaluator.get_bytecode_while_frame_eval_39'
Traceback (most recent call last):
  File "_pydevd_frame_eval/pydevd_frame_evaluator.pyx", line 258, in _pydevd_frame_eval.pydevd_frame_evaluator.get_func_code_info
  File "C:\Users\laimis\AppData\Roaming\Python\Python310\site-packages\debugpy\_vendored\pydevd\pydevd_file_utils.py", line 885, in get_abs_path_real_path_and_base_from_frame
    ret = get_abs_path_real_path_and_base_from_file(f)
  File "C:\Users\laimis\AppData\Roaming\Python\Python310\site-packages\debugpy\_vendored\pydevd\pydevd_file_utils.py", line 852, in get_abs_path_real_path_and_base_from_file
    abs_path, canonical_normalized_filename = _abs_and_canonical_path(f)
  File "C:\Users\laimis\AppData\Roaming\Python\Python310\site-packages\debugpy\_vendored\pydevd\pydevd_file_utils.py", line 411, in _abs_and_canonical_path
    real_path = _apply_func_and_normalize_case(filename, os_path_real_path, isabs, normalize)
  File "C

SystemError: <function Session.__init__ at 0x00000242EBC4EA70> returned NULL without setting an exception

In [35]:
## Info apie viena recepta

def extract_recipe_ingredients(url: str) -> str:
    ## Ingredientai
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    all_ingredients: List[Dict] = []

    recipe_title = soup.find("div", class_="recipe-head").find("h1").text.strip()
    recipe_image = soup.find("div", class_="image").find("img").get("src")
    recipe_ingredients = soup.find("ul", class_="ingredients").find_all("li")
    for ingredient in recipe_ingredients:
        ingredient_span = ingredient.find_all("span")
        all_ingredients.append({            
                "ingredient_name": ingredient_span[0].text.strip(),
                "ingredient_amount": ingredient_span[2].text.strip(),
                "ingredient_note": ingredient_span[3].text.strip()
            })
       
    return str(all_ingredients)


def extract_making_steps(url: str) -> str:

    ## Paruosimo budas
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    recipe_manual: List[str] = []
    recipe_making_steps = soup.find("div", class_="description text").find_all("p")

    for step in recipe_making_steps:
        step_to_txt = step.text.strip()
        recipe_manual.append(step_to_txt)
    
    return "\n".join(recipe_manual) 


In [10]:
## IKEA search products

# sudeda paieskos rezultatos is pirmo psl

def search_products(url) -> List[Dict]:
    """Function for doing search in ikea.lt"""
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    products: List[Dict] = []

    search_results_all = soup.find_all("div", class_="col-6 col-md-4 col-lg-3 p-0 itemBlock")
    for product in search_results_all:
        link = product.find("div", class_="itemInfo").a.get("href")
        products.append({            
                    "product_name": product.find("div", class_="itemInfo").a.text.strip(),
                    "product_info": product.find("div", class_="itemInfo").find("div", class_ = "itemPrice-wrapper").span.text.replace(" €", ""),
                    "product_link": f"https://www.ikea.lt{link}",
                    "product_image": product.find("div", class_ = "productImg").img.get("src")
                })

    return products
    
#prod = search_products("https://www.ikea.lt/lt/search/?q=kede")
#prod

In [5]:
## IKEA retrieve links

from pydantic import BaseModel

__items_per_page__: int = 40
__domain__: str = "https://www.ikea.lt"


class Furniture(BaseModel):
    """Class for base properties of furniture"""
    furniture_name: str
    furniture_description: str
    furniture_price = int
    furniture_image_link: str
    furniture_key_features: str
    furniture_size: str
    
    #furniture_stock_in_store: int    
class FurnitureLink(BaseModel):
    url: str


def _get_page_content(query: str) -> Optional[BeautifulSoup]:
        """Method to get needed content from search result page or whatever related page."""
        resp = requests.get(query)
        if resp.status_code == 200:
            return BeautifulSoup(resp.content)
        raise Exception("Cannot get content. Site is unreachable.")



#################################################

def _retrieve_item_links2(results_count: int, keyword: str) -> List[FurnitureLink]: # prideti self
    """Method to search furnitures by keyword and save specifed number of results."""
    results: List[str] = []  #[FurnitureLink] = []        
    pages_to_iterate: int = (results_count // __items_per_page__) + 1 # kiek paieskos rezultatu puslapiu naudoti        
    
    for page_num in range(1, pages_to_iterate + 1):
        content = _get_page_content(f"{__domain__}/lt/search/?q={keyword}&page={page_num}") #pries metoda prideti self
        pages_count = content.find("ul", class_="pagination mb-0")
        no_results = content.find("div", class_="col mt-4")
        
        if pages_count != None:
            max_number_of_pages = int(content.find("ul", class_="pagination mb-0").find_all("li", class_="page-item")[3].text)
        elif no_results != None:           
            max_number_of_pages = 0
        else:
            max_number_of_pages = 1
        
        if content:
            if max_number_of_pages >= pages_to_iterate:                        
                all_items_per_page = content.find_all("div", class_="col-6 col-md-4 col-lg-3 p-0 itemBlock")
                counter = 1
                
                while results_count >= counter:                                                                    
                    for item in all_items_per_page:
                        link = item.find("div", class_="itemInfo").a.get("href")
                        item = f"{__domain__}{link}"
                        results.append(item)  #(FurnitureLink(url = item))
                        counter += 1                        
                    
            else: 
                print(f"Search has less results than requested.")
                break
        if not content:
            break

    return results


def _retrieve_item_links(results_count: int, keyword: str) -> List[FurnitureLink]:
        """Method to search furnitures by keyword and save specifed number of results."""
        results: List[FurnitureLink] = []        
        pages_to_iterate = (results_count // __items_per_page__) + 1 # kiek paieskos rezultatu puslapiu naudoti        
        
        for page_num in range(1, pages_to_iterate + 1):
            content = _get_page_content(f"{__domain__}/lt/search/?q={keyword}&page={page_num}")
            pages_count = content.find("ul", class_="pagination mb-0")
            no_results = content.find("div", class_="col mt-4")
            
            if pages_count != None:
                max_number_of_pages = int(content.find("ul", class_="pagination mb-0").find_all("li", class_="page-item")[3].text)
            elif no_results != None:           
                max_number_of_pages = 0
            else:
                max_number_of_pages = 1

            if content:
                if max_number_of_pages >= pages_to_iterate:                        
                    all_items_per_page = content.find_all("div", class_="col-6 col-md-4 col-lg-3 p-0 itemBlock")
                    counter = 1

                    while results_count >= counter:                                                 
                        for item in all_items_per_page:
                            link = item.find("div", class_="itemInfo").a.get("href")
                            item = f"{__domain__}{link}"                                 
                            results.append(FurnitureLink(url = item))
                            counter += 1
                else: 
                    print(f"Search has less results than requested.")
                    break
            if not content:
                break

        return results
    

In [7]:
test = _retrieve_item_links(1, "ledai")
links = FurnitureLink()





TypeError: __init__() takes exactly 1 positional argument (2 given)

In [11]:
## IKEA get key features

def _extract_key_features(content: BeautifulSoup) -> str:  # pridet self
    results: List[str] = []   
    features = content.find("div", class_="tab-pane_box").find_all("p")
    
    for feature in features: 
        feature_line = feature.text.strip()
        results.append(feature_line)       
        
    return results

In [122]:
def _extract_furniture_size(content: BeautifulSoup) -> str: #pridet 
    results: List[str] = []
    measures = content.find_all("div", class_="tab-pane_box")[1].find_all("tr") #.find("table")
    
    for measure in measures:
        measure_line = measure.text.strip()
        results.append(measure_line)
    return results

In [2]:
## IKEA extract main info
#resp = requests.get("https://www.ikea.lt/lt/products/valgomasis/sedimieji-valgomojo-baldai/kedes-ir-suoliukai/terje-sulankstomoji-kede-sviesi-pilka-balta-art-80456982")
#resp = requests.get("https://www.ikea.lt/lt/products/ikea-verslui/nekilnojamasis-turtas/sofos/jattebo-dviviete-moduline-sofa-tamsi-geltona-zalia-spr-29471405")
resp = requests.get("https://www.ikea.lt/lt/products/miegamasis/sistema-pax/pakabai-kiti-drabuziu-avalynes-priedai/omtanksam-batu-saukstas-antracito-sp-art-70378070")
content = BeautifulSoup(resp.content, "html.parser")



##############################


def _retrieve_furniture_info(link: str) -> Optional[Furniture]: #prideti self, link: FurnitureLink prie attr
        """Method to get main info about furniture."""
        content = _get_page_content(link.url)  # prideti self prie get metodo

        if content:
            try:
                furniture_name = content.find("div", class_="d-flex align-items-center flex-wrap").h3.text
                furniture_description = content.find("h4", class_="itemFacts font-weight-normal").span.text
                furniture_price = int(content.find("div", class_="itemPrice-wrapper").p.span.text.replace(" €", ""))
            except AttributeError:
                return None

            try:
                furniture_image_link = content.find("a", class_ = "slideImg").find("img").get("src")
            except KeyError:
                furniture_image_link = None

            return Furniture(
                furniture_name = furniture_name,
                furniture_description = furniture_description,
                furniture_price = furniture_price,
                furniture_image_link = furniture_image_link,
                furniture_key_features = _extract_key_features(content), # self
                furniture_size = _extract_furniture_size(content), # self

                #furniture_stock_in_store = int,                
            )
        else:
            return None

NameError: name 'Furniture' is not defined

In [3]:
## pabandyt su selenium, nes reikia hitint mygtuka

from selenium.webdriver.common.by import By

def _check_number_in_stock(content: BeautifulSoup) -> str: # self
    quantity = content.find("div", class_="message noCursor d-flex").find("p").text
    driver = webdriver.Chrome(options=options,chrome_options=chrome_options)
    driver.implicitly_wait(10)
    driver.get(url)
    #time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

In [1]:
## SELENIUM
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# pip install webdriver-manager
from selenium.webdriver.chrome.service import Service # del netinkamos driverio versijos galima autoamtiskai sutvarkjyt
#from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By


# run Selenium in the background by configuring webdriver
options = webdriver.ChromeOptions()
options.headless = True # hide GUI
options.add_argument("start-maximized") # open gui in a full screen mode
options.add_experimental_option("excludeSwitches", ["enable_automation"])
options.add_experimental_option("detach", True)


# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    # this will disable image loading
    "prefs", {"profile.managed_default_content_settings.images": 2}
)


url = "https://www.nordpoolgroup.com/en/Market-data1/Dayahead/Area-Prices/LT/Hourly/?view=table"
driver = webdriver.Chrome(options=options,chrome_options=chrome_options)
#driver.implicitly_wait(10)
driver.get(url)
time.sleep(5)
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')

check = soup.find("div", id="error-content").h1.text
print(check)



#driver.close()
# nepamirst driver.close() or driver.quit()





C:\Users\laimis\AppData\Local\Temp\ipykernel_9112\467024139.py:30: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(options=options,chrome_options=chrome_options)


AttributeError: 'NoneType' object has no attribute 'h1'

In [1]:
### NORDPOOL - METHODS TO GET PAGE CONTENT USING SELENIUM AND RETURN AS SOUP

# Needs a proper method to sleep during execution

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# pip install webdriver-manager
from selenium.webdriver.chrome.service import Service # del netinkamos driverio versijos galima autoamtiskai sutvarkjyt
#from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
from typing import Optional

from selenium.webdriver.common.by import By


country: str = "LT"
query: str = f"https://www.nordpoolgroup.com/en/Market-data1/Dayahead/Area-Prices/{country}/Hourly/?view=table"


def _get_page_content(query: str) -> Optional[BeautifulSoup]:
        """Method to get needed content from search result page or whatever related page."""
        
        chrome_options = webdriver.ChromeOptions()
        # this will disable image loading
        chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
        
        driver = webdriver.Chrome(chrome_options=chrome_options)   #options=options,    
        driver.get(query)
        time.sleep(3)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        checker = soup.find("div", id="error-content")
                
        
        if checker is None:            
            return soup
        elif checker.h1.text == "Important information":
            print("Wrong input.")
            
        driver.quit()
        






#_get_page_content(query)








In [2]:
### MYSQL METHODS

import os
import datetime
import mysql.connector


def _push_to_database(mysql_query: str, params: tuple = None) -> None:
    """Methods retrieves year id from database. If nno such year in database returns NONE."""
    try:
        conn = mysql.connector.connect(
        user=os.environ.get("DB_USER"),
        password=os.environ.get("DB_PASSWORD"),
        database=os.environ.get("DATABASE"),
        port=int(os.environ.get("DB_PORT")),
        host=os.environ.get("DB_HOST"),
        auth_plugin=os.environ.get("AUTH_PLUGIN")
        ) 

        cur = conn.cursor(prepared=True)        
        cur.execute(mysql_query, params)
        conn.commit()
        print("It seems that records inserted succesfully")
        

    except mysql.connector.Error as error:
        print("Failed to execute MySQL query {}".format(error)) # VIETA LOGINIMUI

    finally:
        if conn.is_connected():
            cur.close()
            conn.close()
            print("MySQL conn is closed.") #VIETA LOGINIMUI



def _check_date_in_database(params: str) -> bool:
    """Method checks if date exists in database. Returns boolean."""

    try:
        conn = mysql.connector.connect(
        user=os.environ.get("DB_USER"),
        password=os.environ.get("DB_PASSWORD"),
        database=os.environ.get("DATABASE"),
        port=int(os.environ.get("DB_PORT")),
        host=os.environ.get("DB_HOST"),
        auth_plugin=os.environ.get("AUTH_PLUGIN")
        ) 

        cur = conn.cursor(prepared=True)
        mysql_query: str = """SELECT date_of_price FROM nordpool.HourlyPricesLT where date_of_price=%s;"""
        cur.execute(mysql_query, (params,))
        cur.fetchone()
        row_count = cur.rowcount
        if row_count > 0:       
            return True
        return False  
        
    except mysql.connector.Error as error:
        print("Failed to execute MySQL query {}".format(error)) # VIETA LOGINIMUI

    finally:
        if conn.is_connected():
            cur.close()
            conn.close()
            print("MySQL conn is closed.") #VIETA LOGINIMUI


def _testas(sql_query: str, params: tuple) -> tuple:
    """"""

    try:
        conn = mysql.connector.connect(
        user=os.environ.get("DB_USER"),
        password=os.environ.get("DB_PASSWORD"),
        database=os.environ.get("DATABASE"),
        port=int(os.environ.get("DB_PORT")),
        host=os.environ.get("DB_HOST"),
        auth_plugin=os.environ.get("AUTH_PLUGIN")
        ) 

        cur = conn.cursor(prepared=True)
        #cur.execute('''SELECT NOT EXISTS (SELECT * FROM nordpool.HourlyPricesLT where date_of_price=DATE('%s'));''' %(params))            
        cur.execute(sql_query, params)
        for i in cur:
            return i
               

    except mysql.connector.Error as error:
        print("Failed to execute MySQL query {}".format(error)) # VIETA LOGINIMUI

    finally:
        if conn.is_connected():
            cur.close()
            conn.close()
            print("MySQL conn is closed.") #VIETA LOGINIMUI






"""sql_query = '''SELECT lt_id FROM nordpool.HourlyPricesLT where date_of_price=%s;''' %(today)
        cur.execute(sql_query)
        results = cur.fetchall()
        for result in results:
            year_id = int(result[0])
            return year_id
"""
    

"sql_query = '''SELECT lt_id FROM nordpool.HourlyPricesLT where date_of_price=%s;''' %(today)\n        cur.execute(sql_query)\n        results = cur.fetchall()\n        for result in results:\n            year_id = int(result[0])\n            return year_id\n"

In [61]:
from datetime import date, datetime

data: str = "2022-12-19"
check = _check_date_in_database(data)
print(check)

#print(testas)
#print(type(testas))

MySQL conn is closed.
False


In [5]:
## NORD POOL METHODS TO RETRIEVE DATA AND PUSH IT TO DB

# pip install mysql-connector-python
import os
import mysql.connector
from typing import List, Dict, Optional
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# pip install webdriver-manager
from selenium.webdriver.chrome.service import Service # del netinkamos driverio versijos galima autoamtiskai sutvarkjyt
#from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, date
from bs4 import BeautifulSoup


#poetry add python-dotenv - cia iadziant su jupiter
#from dotenv import load_dotenv


def get_current_prices(soup = _get_page_content(query)) -> None:
    """Populate database with current data in main page."""

    dates = soup.find("tr", class_="column-headers").find_all("th")
    prices = soup.find("table", id="datatable").tbody.find_all("tr")
    counter: int = len(dates) # stulpeliu su datomis skaicius
    
    # Ripping dates off from the site     
    for day_date in reversed(dates[1:counter]): 
        # Pakonvertuoja is 08-12-2022 i 2022-12-08 + pavercia i date tipa
        date_object = datetime.strptime(day_date.text, '%d-%m-%Y').date()            
        price_tags: List = [float]
        exists: bool = _check_date_in_database(date_object)

        if not exists:          
            for priceline in prices[:24]:            
                price_tag = float(priceline.find_all("td")[counter - 1].text.replace(',', '.'))                      
                price_tags.append(price_tag)
                    
            mysql_query: str = '''INSERT INTO nordpool.HourlyPricesLT (date_of_price, time_00_01, time_01_02, time_02_03, time_03_04, time_04_05, time_05_06, time_06_07, time_07_08, time_08_09, time_09_10, time_10_11, time_11_12, time_12_13, time_13_14, time_14_15, time_15_16, time_16_17, time_17_18, time_18_19, time_19_20, time_20_21, time_21_22, time_22_23, time_23_24)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''
            params: tuple = (date_object, price_tags[1], price_tags[2], price_tags[3], price_tags[4], price_tags[5], price_tags[6], price_tags[7], price_tags[8], price_tags[9], price_tags[10], price_tags[11], price_tags[12], price_tags[13], price_tags[14], price_tags[15], price_tags[16], price_tags[17], price_tags[18], price_tags[19], price_tags[20], price_tags[21], price_tags[22], price_tags[23], price_tags[24])
            _push_to_database(mysql_query, params)
            #counter -= 1
        else:
            #counter -= 1
            print(f"Date {date_object} already in.") #loginimui
        counter -= 1
    



test2 = get_current_prices()
test2






C:\Users\laimis\AppData\Local\Temp\ipykernel_9892\17449903.py:28: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)   #options=options,


MySQL conn is closed.
Date 2022-12-30 already in.
MySQL conn is closed.
Date 2022-12-31 already in.
MySQL conn is closed.
Date 2023-01-01 already in.
MySQL conn is closed.
Date 2023-01-02 already in.
MySQL conn is closed.
Date 2023-01-03 already in.
MySQL conn is closed.
Date 2023-01-04 already in.
MySQL conn is closed.
Date 2023-01-05 already in.
MySQL conn is closed.
It seems that records inserted succesfully
MySQL conn is closed.


In [102]:

dayy = "07-12-2022"
date_object = datetime.strptime(dayy, '%d-%m-%Y').date()        

day_id: int = date_object.day

print(day_id)

7
